## Adapting Stackix to work with unpaired Miracle data

In [1]:
import autoencodix as acx
import os
from autoencodix.configs.stackix_config import StackixConfig
from autoencodix.configs.default_config import DataConfig, DataInfo, DataCase
os.chdir("..")
os.getcwd()


/Users/maximilianjoas/development/autoencodix_package/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'/Users/maximilianjoas/development/autoencodix_package'

In [2]:
import os

root_dir = os.path.join("data/miracle/Daten")
rna_file = "2025-05-16_RESTING_RNA_log2TPM_gene_name.parquet"
clin_file = "2025-05-16_RESTING_clinical.parquet" # TODO test with non filtered version including nans -> funktioniert :) 
mut_file = "2025-05-16_RESTING_WES_MutMatrix.parquet"
radio_con = "2025-05-16_RESTING_RADIO_contrast.parquet"
cell_deco = "2025-05-26_RESTING_celldeco.parquet"
clin_file_nas = "2025-05-16_RESTING_clinical.parquet"

# rna_file = "2025-05-16_RESTING_RNA_log2TPM_gene_name.parquet"
# clin_file = "2025-05-16_RESTING_clinical_noNAs.parquet"
# mut_file = "2025-05-16_RESTING_WES_MutMatrix.parquet"
# radio_con = "2025-05-16_RESTING_RADIO_contrast.parquet"
# cell_deco = "2025-05-26_RESTING_celldeco.parquet"

# Create a dictionary of DataInfo objects
data_info = {
    "RNA": DataInfo(file_path=os.path.join(root_dir, rna_file)),
    "clinical": DataInfo(file_path=os.path.join(root_dir, clin_file), data_type="ANNOTATION"),
    "mutation": DataInfo(file_path=os.path.join(root_dir, mut_file)),
    "radiology": DataInfo(file_path=os.path.join(root_dir, radio_con)),
    "cell_deconvolution": DataInfo(file_path=os.path.join(root_dir, cell_deco))
}

config = StackixConfig(
    data_config=DataConfig(
        data_info=data_info
    ),
    data_case=DataCase.MULTI_BULK,
    requires_paired=False,
	# checkpoint_interval=1,
	k_filter=300,
	latent_dim=8,
	train_ratio=0.8,
	valid_ratio=0.1,
	test_ratio=0.1,
    epochs = 200
)

stackix = acx.Stackix(config=config)

In [3]:
stackix.run()

FileNotFoundError: [Errno 2] No such file or directory: 'data/miracle/Daten/2025-05-16_RESTING_RNA_log2TPM_gene_name.parquet'

In [ ]:
stackix.show_result()

In [ ]:
stackix.result.datasets.train.datasets['RNA'].metadata

In [ ]:
param_list = ['SEX', 'DFS_STATUS_3Y']

stackix._visualizer.show_latent_space(result=stackix.result, plot_type="2D-scatter", split='all', param=param_list)

In [ ]:
## Evaluation
from sklearn import linear_model
sklearn_ml_class = linear_model.LogisticRegression(
							solver="sag",
							n_jobs=-1,
							class_weight="balanced",
							max_iter=200,
) ## Support of all sklearn-like models
sklearn_ml_regression = linear_model.LinearRegression()
own_metric_class = 'roc_auc_ovo'  ## sklearn name of metrics: https://scikit-learn.org/stable/modules/model_evaluation.html#string-name-scorers
own_metric_regression = 'r2' 

stackix.evaluate(
	ml_model_class=sklearn_ml_class, # Default is sklearn LogisticRegression
	ml_model_regression=sklearn_ml_regression, # Default is sklearn LinearRegression
	params= param_list,	# No default? ... or all params in annotation?
	metric_class = own_metric_class, # Default is 'roc_auc_ovo'
	metric_regression = own_metric_regression, # Default is 'r2'
	reference_methods = ["PCA", "UMAP", "RandomFeature"], # Default [], no references? or all possible? or only PCA?
	split_type = "use-split", # Default is "use-split", other options: "CV-5", ... "LOOCV"?

)


In [ ]:
# barplot of evaluation results
stackix._visualizer.show_evaluation(
	param = param_list[1], # Plot one parameter
	metric = own_metric_class, 
	ml_alg=str(sklearn_ml_class), # Call via string of ML method
)

In [ ]:
stackix._datasets.train.paired_sample_ids

In [ ]:
# xmodalix = acx.XModalix(config=config)

In [ ]:
results = stackix._trainer._orchestrator.modality_results

In [ ]:
results["RNA"].sample_ids.get(epoch=-1, split="train")

In [ ]:
# xmodalix.run()

In [ ]:

# os.getcwd()